## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula
from datetime import datetime, timedelta

## Read-in

In [2]:
df = pd.read_csv('Houston 9% LIHTC proposed developments - Sheet1.csv')

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [5]:
df.columns

Index(['Developer', 'Project Name', 'Address/Location', 'Total Units',
       'Target Population'],
      dtype='object')

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [7]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Address/Location'].apply(geocode)

In [8]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [9]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

## HTML Popup Formatter

In [11]:
df.columns

Index(['Developer', 'Project Name', 'Address/Location', 'Total Units',
       'Target Population', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [12]:
def popup_html(row):
    developer = row['Developer']
    name = row['Project Name']
    address = row['Address/Location']
    num_units = row['Total Units']
    target_pop = row['Target Population']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Developer: </strong>{}'''.format(developer) + '''<br>
    <strong>Project Name: </strong>{}'''.format(name) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Number of Units: </strong>{}'''.format(num_units) + '''<br>
    <strong>Target Population: </strong>{}'''.format(target_pop) + '''<br>
    </html>
    '''
    return html

In [20]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9,tiles=None)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Houston 9% LIHTC proposed developments')


for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)


folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [20]:
m.save('index.html')

## Data snagger

In [21]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [22]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
The Setai, 101 20th St 2507 Miami | Price $4,150,000 | $3,245 psf | Listing agent: Diana Garchitorena with Douglas Elliman | Buyer's agent: Jeffrey Miller with One Sotheby's International Realty | Days on market: 88
Least Expensive
Atlantis on Brickell, 2025 Brickell Ave 1401 | Price $1,200,000 | $558 psf | Listing agent: Saverina Martinez PA with Galleria International Realty | Buyer's agent: Maria Lopez Pulido with Compass Florida, LLC. | Days on market: 225
Highest Price Per Square Foot
The Setai, 101 20th St 2507 Miami | Price $4,150,000 | $3,245 psf | Listing agent: Diana Garchitorena with Douglas Elliman | Buyer's agent: Jeffrey Miller with One Sotheby's International Realty | Days on market: 88
Lowest Price Per Square Foot
7000 Island Blvd Williams Island, 7000 Island Blvd 2502 | Price $1,285,000 | $464 psf | Listing agent: David Salazar with Affinity International Realty | Buyer's agent: Nicole Shein with Williams Island Realty, LLC | Days on market: 6
Most Days 

In [23]:
df['days_on_market'].mean()

88.1

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [24]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_02242024


# CREATE TEMPLATE 

In [25]:
df['muni_name'] = df['building_address_city'].str.split('\r').str[-1]

In [26]:
muni_set = set(df['muni_name'])

In [27]:
df['building_address_city_clean'] = df['building_address_city'].str.split('\r')

In [28]:
top_sale = df.at[1, 'building_address_city_clean']
second_top_sale = df.at[2,'building_address_city_clean']

In [29]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    

### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [30]:
difference

579762.45

In [31]:
story_string = f'''
\033[1mHED:\033[0m {df.loc[df['int_Sale_Price'].idxmax()]['building_name']} tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from {df.at[10,'Sale Price']} to {df.at[1,'Sale Price']}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Credit condo.com with the image below and the PDF chart in the story
{top_condo_image_url}

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from condo.com. Condos closed after an average of {average_days_on_market} days on the market.

For the top 10 sales, prices ranged from {df.at[10,'Sale Price']} to {df.at[1,'Sale Price']}. 

{top_sale[0]}, at {top_sale[1]} in {top_sale[2]} took the top spot with a {df.at[1,'Sale Price']} million closing. {df.at[1,'Agent']} with {df.at[1,'Listing Broker']} had the listing. {df.at[1,'Buyer Agent']} with {df.at[1,'Buyer Broker']} represented the buyer. The sale closed after {df.at[1,'days_on_market']} days on the market at ${df.at[1,'price_per_sqft']:,} per square foot.

{second_top_sale[0]} in {second_top_sale[2]}, at {second_top_sale[1]}, closed for the second highest amount, {df.at[2,'Sale Price']}, or ${df.at[2,'price_per_sqft']:,} per square foot, after {df.at[2,'days_on_market']} days on the market. {df.at[2,'Agent']} with {df.at[2,'Listing Broker']} had the listing. {df.at[2,'Buyer Agent']} with {df.at[2,'Buyer Broker']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

# print(story_string)

In [32]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
    
3. Add in a screenshot of the sales chart.
'''

In [33]:
print(story_string)

### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

print(story_checklist)


HED: The Setai tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,200,000.00 to $4,150,000.00
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Credit condo.com with the image below and the PDF chart in the story
https://www.condo.com/building/The-Setai-Miami-Beach-FL-33139-3850835

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Aventura, Miami Beach, Sunny Isles Beach, Miami, Coral Gables

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 100 condo sales totaling $60,776,400 million from Feb. 18th to Feb. 24th. The previous week, brokers closed 141 condo sales totaling $167,441